In [ ]:
#!pip install crewai==0.28.8
#!pip install crewai_tools==0.1.6
#!pip install langchain_community==0.0.29
#!pip install langchain_openai
#!pip install openai

In [ ]:
import os
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process #Import the required classes from the crewAI libray.
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool #Import crewAI tools that will allow the agent to perform search (google, websites) and scrape websites.

In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
#OpenAI API.
#openai_api_key = 'ADD HERE YOUR API KEY'
#openai_model_name = 'gpt-3.5-turbo'
#openai_api_base = 'https://api.openai.com/v1'
#llm = ChatOpenAI(openai_api_key = openai_api_key, openai_api_base = openai_api_base, model_name = openai_model_name)

#Groq API.
groq_api_key = 'ADD HERE YOUR API KEY'
model_name = 'llama-3.3-70b-versatile'
api_base = 'https://api.groq.com/openai/v1'
llm = ChatOpenAI(openai_api_key = groq_api_key, openai_api_base = api_base, model_name = model_name)

In [ ]:
#Create the Data Analyst Agent and assign the search and scrape tools.
data_analyst_agent = Agent(
    role = "Data Analyst",
    goal = "Monitor and analyze market data in real-time to identify trends and predict market movements.",
    backstory = "Specializing in financial markets, this agent uses statistical modeling and machine learning to provide crucial insights."
								" With a knack for data, the Data Analyst Agent is the cornerstone for informing trading decisions.",
    verbose = True,
    allow_delegation = True,
    tools = [scrape_tool, search_tool],
		llm = llm,
)

In [ ]:
#Create the Trading Strategy Agent and assign the search and scrape tools.
trading_strategy_agent = Agent(
    role = "Trading Strategy Developer",
    goal = "Develop and test various trading strategies based on insights from the Data Analyst Agent.",
    backstory = "Equipped with a deep understanding of financial markets and quantitative analysis, this agent devises and refines trading strategies."
								" It evaluates the performance of different approaches to determine the most profitable and risk-averse options.",
    verbose = True,
    allow_delegation = True,
    tools = [scrape_tool, search_tool],
		llm = llm,
)

In [ ]:
#Create the Execution Agent and assign the search and scrape tools.
execution_agent = Agent(
    role = "Trade Advisor",
    goal = "Suggest optimal trade execution strategies based on approved trading strategies.",
    backstory = "This agent specializes in analyzing the timing, price, and logistical details of potential trades."
                " By evaluating these factors, it provides well-founded suggestions for when and how trades should be executed to maximize efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = llm,
)

In [ ]:
#Create the Risk Management Agent and assign the search and scrape tools.
risk_management_agent = Agent(
    role = "Risk Advisor",
    goal = "Evaluate and provide insights on the risks associated with potential trading activities.",
    backstory = "Armed with a deep understanding of risk assessment models and market dynamics, this agent scrutinizes the potential risks of proposed trades."
              " It offers a detailed analysis of risk exposure and suggests safeguards to ensure that trading activities align with the firm’s risk tolerance.",
    verbose = True,
    allow_delegation = True,
    tools = [scrape_tool, search_tool],
    llm = llm,
)

In [ ]:
#Define the Data Analyst Agent task, to analyse market data.
data_analysis_task = Task(
    description = (
                  "Continuously monitor and analyze market data for the selected stock ({stock_selection}). "
                  "Use statistical modeling and machine learning to identify trends and predict market movements."
                  ),
    expected_output = (
                      "Insights and alerts about significant market opportunities or threats for {stock_selection}."
                      ),
    agent = data_analyst_agent,
)

In [ ]:
#Define the Trading Strategy Agent task, to develop trading strategies.
strategy_development_task = Task(
    description = (
                  "Develop and refine trading strategies based on the insights from the Data Analyst and user-defined risk tolerance ({risk_tolerance}). "
                  "Consider trading preferences ({trading_strategy_preference})."
                  ),
    expected_output = (
                      "A set of potential trading strategies for {stock_selection} that align with the user's risk tolerance."
                      ),
    agent = trading_strategy_agent,
)

In [ ]:
#Define the Execution Agent task, to plan trade execution.
execution_planning_task = Task(
    description = (
                  "Analyze approved trading strategies to determine the best execution methods for {stock_selection}, "
                  "considering current market conditions and optimal pricing."
                  ),
    expected_output = (
                      "Detailed execution plans suggesting how and when to execute trades for {stock_selection}."
                      ),
    agent = execution_agent,
)

In [ ]:
#Define the Risk Management Agent task, to asses the trading risks.
risk_assessment_task = Task(
    description = (
                  "Evaluate the risks associated with the proposed trading strategies and execution plans for {stock_selection}. "
                  "Provide a detailed analysis of potential risks and suggest mitigation strategies."
                  ),
    expected_output = (
                      "A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_selection}."
                      ),
    agent = risk_management_agent,
)

In [ ]:
#Create the Crew of agents.
financial_trading_crew = Crew(
    agents = [data_analyst_agent,
              trading_strategy_agent,
              execution_agent,
              risk_management_agent],

    tasks = [data_analysis_task,
            strategy_development_task,
            execution_planning_task,
            risk_assessment_task],

    manager_llm = llm, #Define a manager to manage and delegate tasks to agents. The manager is created automatically by CrewAI.
    process = Process.hierarchical, #Set the crew to operate hierarchically, meaning that the agents will ¨report¨ to one manager.
    verbose = True
)

In [ ]:
#Define your financial inputs and trigger the chain of tasks.
financial_trading_inputs = {
    'stock_selection': 'PATH',
    'initial_capital': '10000',
    'risk_tolerance': 'Low',
    'trading_strategy_preference': 'Swing Trading',
    'news_impact_consideration': True
}
result = financial_trading_crew.kickoff(inputs = financial_trading_inputs)

In [ ]:
#Display the results as Markdown, for improved readability.
from IPython.display import Markdown
Markdown(result)